# Sea Ice Advance and Retreat

Using the daily NSIDC sea ice extent product, figure out the day in each year when sea ice is present in each pixel to calculate day of sea ice advance and retreat, and the duration between the two

In [1]:
#directory paths
_work_dir='/g/data/gv90/as2285/miz/'
_data_dir='/g/data/gv90/P6_data/'

In [2]:
#useful py libraries
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from datacube.utils.cog import write_cog
import odc.geo.xr
from dea_tools.spatial import subpixel_contours
from affine import Affine

In [3]:
import sys
sys.path.append(_work_dir)
from utils.sea_ice_duration import sea_ice_duration

# Open the dataset

For this, we are going to treat historical and near real-time data as equivalent.

In [6]:
#lists of both the historical and near real-time files
daily_files = ! ls -d {_data_dir}NSIDC/G02202_V4/seaice_conc_daily*.nc {_data_dir}NSIDC/G10016_V2/daily/*2023*.nc

In [7]:
#open it all
cdr_daily_ds=xr.open_mfdataset(
    daily_files, combine='nested',concat_dim='tdim', data_vars='minimal',
).swap_dims(
    {'tdim':'time', 'x':'xgrid','y':'ygrid'}
).rename(
    {'xgrid':'x','ygrid':'y'}
)  

In [8]:
#maybe change to nasa team
cdr_daily_da=cdr_daily_ds.cdr_seaice_conc.odc.assign_crs("epsg:3976")

/g/data/gv90/as2285/miniconda3/envs/sea-ice-tracker/lib/python3.10/site-packages/odc/geo/_xr_interop.py:307: UserWarning: grid_mapping=projection is not pointing to valid coordinate
  warnings.warn(


# Calculate advance day / retreat day and season

For year of the groups defined (stating/ending Feb 15), find the day when advance and retreat starts:

In [9]:
cdr_duration=sea_ice_duration(cdr_daily_da)

In [10]:
cdr_duration.calc_duration()

In [11]:
adv_day_ds=cdr_duration.adv_day_ds
ret_day_ds=cdr_duration.ret_day_ds
duration_da=cdr_duration.duration_da

In [12]:
cdr_duration.calc_climat()

In [13]:
cdr_duration.calc_anoms()

# Contours for tracker

In [14]:
cdr_duration.duration_da.year.values

array([1979., 1980., 1981., 1982., 1983., 1984., 1985., 1986., 1987.,
       1988., 1989., 1990., 1991., 1992., 1993., 1994., 1995., 1996.,
       1997., 1998., 1999., 2000., 2001., 2002., 2003., 2004., 2005.,
       2006., 2007., 2008., 2009., 2010., 2011., 2012., 2013., 2014.,
       2015., 2016., 2017., 2018., 2019., 2020., 2021., 2022.])

In [16]:
# cdr_duration.duration_da.load()

for i in cdr_duration.duration_da.year.values[-1:]:

    lines=subpixel_contours(
        cdr_duration.duration_da.sel(year=i).load(), 
        z_values=[100,200,300], min_vertices=15, crs='epsg:3976'
    )

    lines.to_crs('epsg:4326').to_file(
        f'{_data_dir}/tracker_data/duration/duration_{int(i)}.json', 
        driver='GeoJSON'
    )

In [17]:
# cdr_duration.adv_day_ds.load()

for i in cdr_duration.adv_day_ds.year.values[-1:]:

    contour_dates=np.array([
        f'{int(i)}-{iMonth:02.0f}-01' for iMonth in [4,6,8] #july, august, sept
    ], dtype=np.datetime64)

    levels=[(iCon-np.datetime64(f'{int(i)}-02-16')).astype(int) for iCon in contour_dates]
    
    lines=subpixel_contours(
        cdr_duration.adv_day_ds.sel(year=i).index.load(), 
        z_values=levels, 
        min_vertices=15, 
        crs='epsg:3976',
        attribute_df=pd.DataFrame(contour_dates, columns=['Date'])
    )

    lines.to_crs('epsg:4326').to_file(
        f'{_data_dir}/tracker_data/advance_day/advance_{int(i)}.json', 
        driver='GeoJSON'
    )

/g/data/gv90/as2285/miniconda3/envs/sea-ice-tracker/lib/python3.10/site-packages/geopandas/io/file.py:545: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


In [18]:
# cdr_duration.ret_day_ds.load()

for i in cdr_duration.ret_day_ds.year.values[-1:]:

    contour_dates=np.array([
        *[np.datetime64(f'{int(i)}-{iMonth:02.0f}-01') for iMonth in [11,12]],
        np.datetime64(f'{int(i)+1}-01-01')
    ], dtype=np.datetime64)

    levels=[(iCon-np.datetime64(f'{int(i)}-02-16')).astype(int) for iCon in contour_dates]
    
    lines=subpixel_contours(
        cdr_duration.ret_day_ds.sel(year=i).index.load(), 
        z_values=levels, 
        min_vertices=15, 
        crs='epsg:3976',
        attribute_df=pd.DataFrame(contour_dates, columns=['Date'])
    )

    lines.to_crs('epsg:4326').to_file(
        f'{_data_dir}/tracker_data/retreat_day/retreat_{int(i)}.json', 
        driver='GeoJSON'
    )

# COGS for tracker

In [19]:
for i in cdr_duration.dur_anom_da.year.values[-1:]:
    
    write_cog(cdr_duration.dur_anom_da.sel(year=i).load(), #consistent 0-100 scale to give %
            f'{_data_dir}tracker_data/duration_anoms_cog/dur_anom_{int(i)}.tiff',
            overwrite=True,
            overview_levels=[2,4,8,16,32]
        )

In [20]:
cdr_duration.adv_anom_da.load()

for i in cdr_duration.adv_anom_da.year.values[-1:]:
    
    write_cog(cdr_duration.adv_anom_da.sel(year=i).load(), #consistent 0-100 scale to give %
            f'{_data_dir}tracker_data/adv_anoms_cog/adv_anom_{int(i)}.tiff',
            overwrite=True,
            overview_levels=[2,4,8,16,32]
        )

In [21]:
cdr_duration.ret_anom_da.load()

for i in cdr_duration.ret_anom_da.year.values[-1:]:
    
    write_cog(cdr_duration.ret_anom_da.sel(year=i).load(), #consistent 0-100 scale to give %
            f'{_data_dir}tracker_data/ret_anoms_cog/ret_anom_{int(i)}.tiff',
            overwrite=True,
            overview_levels=[2,4,8,16,32]
        )